In [141]:
# 4.1 - Explorando o “COVID-19 Dataset”

import numpy as np
import pandas as pd

covid=pd.read_csv("../input/corona-virus-report/covid_19_clean_complete.csv")
covid.head()

In [142]:
# Só pra saber se é o q aconteceu no dia ou é cumulativo
brasil = covid.loc[covid['Country/Region'].str.contains('Brazil')]
brasil

In [120]:
# 1)a)
print('O banco de dados tem',len(covid),'linhas e',len(list(covid)),'colunas')
# 1)b)
print('\nAs colunas são:')
print(list(covid))
# 1)c)
print('\nOs tipos de variável são:')
print(covid.dtypes)
# 1)d)
covid['Date'] = pd.to_datetime(covid['Date'])

In [121]:
# 1)e)
covid.describe(datetime_is_numeric=True)

In [122]:
# 1)g)
covid.isna().any()

In [125]:
# 2)
china = covid.loc[covid['Country/Region'].str.contains('China')]
china5 = china.groupby('Province/State')['Confirmed'].max().nlargest(5)
china5 = list(china5.index.values)

china5df = china.loc[china['Province/State'].str.contains('|'.join(china5))]
china5df = china5df.drop(['Lat','Long','Date','WHO Region'],axis=1)
china5df

In [130]:
# 3)
def faltante(row):
    if str(row['Province/State']) == 'nan':
        return row['Country/Region']
    else:
        return row['Country/Region']+'_'+row['Province/State']
covid2 = covid.copy()
covid2['Country/Region'] = covid2.apply(lambda row: faltante(row),axis=1)
covid2 = covid2.drop(['Province/State'], axis=1)
covid2

In [159]:
# 4)

wmd=pd.read_csv("../input/corona-virus-report/worldometer_data.csv")
wmdF = wmd.drop(['TotalCases','NewCases','TotalDeaths','NewDeaths','TotalRecovered',
                'NewRecovered','ActiveCases','Serious,Critical','Tot Cases/1M pop',
                'Deaths/1M pop','TotalTests','Tests/1M pop','WHO Region'],axis=1)

prov = covid.groupby(['Province/State','Country/Region'], dropna=False)['Deaths'].max()
paises = prov.groupby(['Country/Region']).sum()
covidPpais = wmdF.merge(paises, on='Country/Region')
covidPcont = covidPpais.groupby(['Continent']).sum()
covidPcont['Mortes/Milhão'] = covidPcont.apply(lambda row:
                                               (row['Deaths']/row['Population'])*10**6,
                                               axis=1)
covidPcont.sort_values(by='Mortes/Milhão', ascending=False)